In [1]:
monitor='val_accuracy'
epochs=90
batch_size=32
input_shape=(224, 224, 3) # please resize it to (224,224,3) if you have enough RAM
Verbose=True

In [2]:
#apt-get install git python3-opencv
import os

if not os.path.isdir('k'):
  !git clone https://github.com/hamidpeywasti/keras-neural-api k
else:
  !cd k && git pull

!cd k && pip install .

!rm -rf k

Cloning into 'k'...
remote: Enumerating objects: 1828, done.
remote: Counting objects: 100% (240/240), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 1828 (delta 145), reused 147 (delta 66), pack-reused 1588
Receiving objects: 100% (1828/1828), 15.73 MiB | 7.20 MiB/s, done.
Resolving deltas: 100% (1258/1258), done.
Processing /kaggle/working/k
  Preparing metadata (setup.py) ... done
  Created wheel for cai: filename=cai-0.1.7-py3-none-any.whl size=61387 sha256=03290e504b0278973bb9af1b65588bddbbe322f5848603e094b8c21246c3b555
  Stored in directory: /tmp/pip-ephem-wheel-cache-dphsra_m/wheels/7f/63/1a/ed2b6234f76aae55efe76492246448486e41178bd0b5682ecf
Successfully built cai


In [3]:
import sys
print("Python version:", sys.version)

import skimage
print('skimage:',  skimage.__version__)

import tensorflow as tf
print('Tensorflow:',tf.__version__)

import cai
import cai.datasets
import cai.models

import multiprocessing
import os

from sklearn.metrics import confusion_matrix
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

Python version: 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
skimage: 0.21.0
Tensorflow: 2.13.0


In [4]:
data_dir = "/kaggle/input/apple-leaf-diseases-13-classes"

label_of_classes = os.listdir(data_dir)
print("Classes Lables:", label_of_classes)

number_of_classes = len(label_of_classes)
print("Number of Classes:", number_of_classes)

Classes Lables: ['Powdery Mildew', 'Leaf Blotch', 'Apple rot', 'Block rot', 'Grey Spot', 'Frogeye Leaf Spot', 'Mosaic', 'Alternaria Leaf Spot', 'Healthy', 'Rust', 'Brown Spot', 'Apple Scab', 'Cedar apple rust']
Number of Classes: 13


In [5]:
train_x, val_x, test_x, train_y, val_y, test_y, classweight, classes = cai.datasets.load_images_from_folders(
    seed=7,
    root_dir=data_dir,
    lab=True,
    verbose=Verbose,
    bipolar=False,
    base_model_name='apple13_leaf',
    training_size=0.6,
    validation_size=0.2,
    test_size=0.2,
    target_size=(input_shape[0],input_shape[1]),
    has_training=True,
    has_validation=True,
    has_testing=True,
    smart_resize=True
)

Loading  13  classes.
smart resize is enabled.
loading train images
train shape is: (3516, 224, 224, 3)
loading validation images
validation shape is: (1171, 224, 224, 3)
loading test images
test shape is: (1180, 224, 224, 3)
Channel  0  min: 0.0  max: 1.0
Channel  1  min: 0.0  max: 0.8897463
Channel  2  min: 0.0  max: 0.96768874
Loaded.


In [6]:
print(train_x.shape,val_x.shape,test_x.shape)
print(train_y.shape,val_y.shape,test_y.shape)

(3516, 224, 224, 3) (1171, 224, 224, 3) (1180, 224, 224, 3)
(3516, 13) (1171, 13) (1180, 13)


In [7]:
print(multiprocessing.cpu_count())

4


In [8]:
all_history = []
epochs=1
for two_paths_second_block in [False, True]:
    for l_ratio in [0.1, 0.9]: #[0.1,0.3,0.5,0.7,0.9]:
        
        basefilename = 'Apple13-twoPath-inception-' + str(two_paths_second_block) + '-' + str(l_ratio)
        print('Running: '+basefilename)
          
        model = cai.models.compiled_two_path_inception_v3(
            input_shape=input_shape, 
            classes=number_of_classes, 
            two_paths_first_block=True,
            two_paths_second_block=two_paths_second_block,
            l_ratio=l_ratio,
            ab_ratio=(1-l_ratio),
            max_mix_idx=5, 
            model_name='two_path_inception_v3'
        )
          
        best_result_file_name = basefilename+'-best_result.hdf5'
          
        save_best = tf.keras.callbacks.ModelCheckpoint(
            filepath=best_result_file_name, 
            monitor=monitor, 
            verbose=1,
            save_best_only=True,
            save_weights_only=False, 
            mode='max',
            save_freq='epoch'
        )
          
        history = model.fit(
            train_x,
            train_y, 
            epochs=epochs, 
            batch_size=batch_size,
            validation_data=(val_x,val_y),
            callbacks=[save_best],
            class_weight=classweight,
            workers=multiprocessing.cpu_count()
        )

        all_history.append(history)
          
        print('Testing Last Model: '+basefilename)
          
        evaluated = model.evaluate(test_x,test_y)
          
        for metric, name in zip(evaluated,["loss","acc","top 5 acc"]):
            print(name,metric)
            
        print('Best Model Results: '+basefilename)
          
        model = tf.keras.models.load_model(
            best_result_file_name,
            custom_objects={'CopyChannels': cai.layers.CopyChannels}
        )
          
        evaluated = model.evaluate(test_x,test_y)
          
        cai.models.save_model(model, basefilename)
          
        for metric, name in zip(evaluated,["loss","acc","top 5 acc"]):
          print(name,metric)
        
        print('Finished: '+basefilename)

Running: Apple13-twoPath-inception-False-0.1
110/110 [==============================] - ETA: 0s - loss: 1.3789 - accuracy: 0.5304
Epoch 1: val_accuracy improved from -inf to 0.12041, saving model to Apple13-twoPath-inception-False-0.1-best_result.hdf5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


110/110 [==============================] - 1001s 9s/step - loss: 1.3789 - accuracy: 0.5304 - val_loss: 8.5388 - val_accuracy: 0.1204
Testing Last Model: Apple13-twoPath-inception-False-0.1
37/37 [==============================] - 66s 2s/step - loss: 8.5298 - accuracy: 0.1229
loss 8.52977180480957
acc 0.12288135290145874
Best Model Results: Apple13-twoPath-inception-False-0.1
37/37 [==============================] - 68s 2s/step - loss: 8.5298 - accuracy: 0.1229
loss 8.52977180480957
acc 0.12288135290145874
Finished: Apple13-twoPath-inception-False-0.1
Running: Apple13-twoPath-inception-False-0.9
110/110 [==============================] - ETA: 0s - loss: 1.4555 - accuracy: 0.4647
Epoch 1: val_accuracy improved from -inf to 0.10675, saving model to Apple13-twoPath-inception-False-0.9-best_result.hdf5
110/110 [==============================] - 988s 9s/step - loss: 1.4555 - accuracy: 0.4647 - val_loss: 3.3578 - val_accuracy: 0.1067
Testing Last Model: Apple13-twoPath-inception-False-0.9
37/